In [1]:
!nvidia-smi

Sat Sep 30 06:44:49 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 531.14       CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 950M         On | 00000000:01:00.0 Off |                  N/A |
| N/A    0C    P8               N/A /  N/A|      0MiB /  2048MiB |      1%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
# # Cloning DeepSORT
# !git clone https://github.com/nwojke/deep_sort.git

The original DeepSORT repo uses a deprecated sklearn function called `linear_assignment`, which needs to be replaced for error free execution of code with scipy.



1.   Open ./deep_sort/deep_sort/linear_assignment.py
2.   Replace `from sklearn.utils.linear_assignment_ import linear_assignment` in line 4 with `from scipy.optimize import linear_sum_assignment`.

3.   Replace `indices = linear_assignment(cost_matrix)` in line 58 with the following lines of code:
```
  indices = linear_sum_assignment(cost_matrix)
  indices = np.asarray(indices)
  indices = np.transpose(indices)
```





Also, rename ./deep_sort/tools as ./deep_sort/tools_deepsort to avoid any name overlapping.

In [3]:
# !pip3 install tensorflow

In [4]:
# DeepSORT imports.
%cd ./deep_sort

/mnt/d/BKHN/alpr_project/learnopencv/deep_sort


In [5]:

from application_util import preprocessing
from deep_sort import nn_matching
from deep_sort.detection import Detection
from deep_sort.tracker import Tracker
from tools_deepsort import generate_detections as gdet
import uuid

2023-09-30 06:44:54.105284: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-30 06:44:54.162838: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-30 06:44:54.164562: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-30 06:44:55.193042: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## OCR

##Installing requirements

In [6]:
%cd ../

/mnt/d/BKHN/alpr_project/learnopencv


In [7]:
%ls

ALPR_inference.ipynb*                  bus.jpg*     test1.py*    yolov8/
License_plate_detection_YOLOv4.ipynb*  deep_sort/   test2.py*
OCR_comparison.ipynb*                  model_data/  test_image/
PaddleOCR/                             results/     test_video/


In [8]:
# # No need to install if already installed requirements.txt
# !pip install paddlepaddle-gpu
# !pip install "paddleocr>=2.0.1"

## Importing libraries

In [9]:
from paddleocr import PaddleOCR
ocr = PaddleOCR(lang='en',rec_algorithm='CRNN')

[2023/09/30 06:45:00] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='CRNN', rec_model_dir='/root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, rec_char_dict_path='/usr/l

## Utility functions

## Detector

Continuing process from License plate detection [notebook](https://colab.research.google.com/github/sanyam83/learnopencv/blob/master/ALPR/License_plate_detection_YOLOv4.ipynb). (Assuming the files and weights are now created)

In [10]:
!pwd

/mnt/d/BKHN/alpr_project/learnopencv


In [11]:
%ls

ALPR_inference.ipynb*                  bus.jpg*     test1.py*    yolov8/
License_plate_detection_YOLOv4.ipynb*  deep_sort/   test2.py*
OCR_comparison.ipynb*                  model_data/  test_image/
PaddleOCR/                             results/     test_video/


In [12]:
import cv2
# Variables storing colors and fonts.
font = cv2.FONT_HERSHEY_SIMPLEX
blue_color = (255,0,0)
white_color = (255,255,255)
black_color = (0,0,0)
green_color = (0,255,0)
yellow_color = (178, 247, 218)


## Utility functions

In [13]:
def crop(image, coord):
  # Cropping is done by -> image[y1:y2, x1:x2].
  cr_img = image[int(coord[1]):int(coord[3]), int(coord[0]):int(coord[2])]
  return cr_img

In [14]:
def resize_bbox(detections, out_size, in_size):
  coord = []
  scores = []

  # Scaling the bounding boxes to the different size
  for det in detections:
    points = list(det[2])
    conf = det[1]
    xmin, ymin, xmax, ymax = darknet.bbox2points(points)
    y_scale = float(out_size[0]) / in_size[0]
    x_scale = float(out_size[1]) / in_size[1]
    ymin = int(y_scale * ymin)
    ymax = int(y_scale * ymax)
    xmin = int(x_scale * xmin) if int(x_scale * xmin) > 0 else 0
    xmax = int(x_scale * xmax)
    final_points = [xmin, ymin, xmax-xmin, ymax-ymin]
    scores.append(conf)
    coord.append(final_points)
  return coord, scores

In [15]:
def get_best_ocr(preds, rec_conf, ocr_res, track_id):
  for info in preds:
    # Check if it is current track id
    if info['track_id'] == track_id:
      # Check if the ocr confidenence is maximum or not
      if info['ocr_conf'] < rec_conf:
        info['ocr_conf'] = rec_conf
        info['ocr_txt'] = ocr_res
      else:
        rec_conf = info['ocr_conf']
        ocr_res = info['ocr_txt']
      break
  return preds, rec_conf, ocr_res

#Inference

## Fucntion for inferencing on images

In [16]:
%ls

ALPR_inference.ipynb*                  bus.jpg*     test1.py*    yolov8/
License_plate_detection_YOLOv4.ipynb*  deep_sort/   test2.py*
OCR_comparison.ipynb*                  model_data/  test_image/
PaddleOCR/                             results/     test_video/


In [12]:
from ultralytics import YOLO

# Load a model
# model = YOLO("yolov8/alpr.yaml")  # build a new model from scratch
model = YOLO("yolov8/alpr_yolov8n_8000img_100epochs.pt")  # load a pretrained model

# Use the model
# model.train(data="coco128.yaml", epochs=3)  # train the model
# metrics = model.val()  # evaluate model performance on the validation set
results = model("./test_image/Cars187.png")  # predict on an image
print("results:", results)



image 1/1 /mnt/d/BKHN/alpr_project/learnopencv/test_image/Cars187.png: 384x640 1 plate, 41.0ms
Speed: 5.2ms preprocess, 41.0ms inference, 4.2ms postprocess per image at shape (1, 3, 384, 640)


results: [ultralytics.yolo.engine.results.Results object with attributes:

boxes: ultralytics.yolo.engine.results.Boxes object
keypoints: None
keys: ['boxes']
masks: None
names: {0: 'plate'}
orig_img: array([[[180, 198, 220],
        [180, 198, 220],
        [179, 197, 220],
        ...,
        [253, 230, 239],
        [254, 230, 239],
        [253, 230, 239]],

       [[179, 198, 220],
        [179, 198, 220],
        [178, 197, 219],
        ...,
        [254, 230, 239],
        [254, 230, 239],
        [254, 230, 239]],

       [[177, 195, 217],
        [177, 195, 217],
        [175, 194, 215],
        ...,
        [253, 230, 240],
        [254, 230, 240],
        [253, 230, 239]],

       ...,

       [[ 17,  18,  25],
        [ 17,  17,  24],
        [ 17,  18,  25],
        ...,
        [  8,   9,   8],
        [  7,   9,   7],
        [  7,   9,   7]],

       [[ 13,  13,  14],
        [ 13,  13,  14],
        [ 13,  13,  16],
        ...,
        [  6,   6,   4],
        [  8,

In [90]:
from ultralytics import YOLO
import cv2
import os

# Load a model
model = YOLO("yolov8/alpr_yolov8n_8000img_100epochs.pt") 

def test_img_yolov8(input, out_path):
    file_name = input.split("/")[-1]
    print("file_name:", file_name)
    # Use the model
    results = model(input)  # predict on an image
    img = cv2.imread(input)
    for result in results:
        if result.boxes is not None and len(result.boxes) > 0:
            # Get the bounding boxes and image for each result
            bboxes = result.boxes[0].cpu().numpy()
            # img = cv2.imread(input)  # Đảm bảo bạn đã định nghĩa biến 'input'
    
            # Loop through bboxes and apply OCR, then draw on the image
            for bbox in bboxes:
                xyxy = bbox.xyxy
                x1, y1, x2, y2 = xyxy[0]
    
                # Kiểm tra xem biển số xe có gần vuông không (ví dụ: tỷ lệ 1:1)
                width = x2 - x1
                height = y2 - y1
                aspect_ratio = width / height
                print("aspect_ratio:", aspect_ratio)
                
# ##################################
#                 cr_img = img[int(y1):int(y2), int(x1):int(x2)]
#                 # Chuyển ảnh sang độ xám để phát hiện cạnh
#                 gray = cv2.cvtColor(cr_img, cv2.COLOR_BGR2GRAY)
                
#                 # Sử dụng phát hiện cạnh (Canny) để tạo ra ảnh cạnh
#                 edges = cv2.Canny(gray, threshold1=50, threshold2=150, apertureSize=3)
                
#                 # Tìm các đường thẳng trong ảnh
#                 lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=100, minLineLength=100, maxLineGap=10)
                
#                 # Xác định góc nghiêng của ảnh
#                 if lines is not None:
#                     angles = []
#                     for line in lines:
#                         x1, y1, x2, y2 = line[0]
#                         angle = np.degrees(np.arctan2(y2 - y1, x2 - x1))
#                         angles.append(angle)
                
#                     median_angle = np.median(angles)
                
#                     # Xoay lại ảnh để đưa góc nghiêng về 0 độ
#                     img_rotate = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE if median_angle < 0 else cv2.ROTATE_90_COUNTERCLOCKWISE)
#                     image_filename = str(file_name) + "_rotate.jpg"  # Tên file ảnh đầu ra
#                     cv2.imwrite("results/crop_image/" + image_filename, image_collage_horizontal)
#                 # else:
#                 #     # Không tìm thấy đường thẳng, không xoay ảnh
#                 #     img = img
##############################
                if 0 <= aspect_ratio <= 1.5:
                    # Biển số xe gần vuông hoặc hình vuông
    
                    # Tính toán điểm chia ảnh thành hai phần trên và dưới
                    split_point = y1 + (y2 - y1) // 2

                    
                    # Tạo hai phần ảnh con từ ảnh cr_img
                    upper_part = img[int(y1):int(split_point), int(x1):int(x2)]
                    lower_part = img[int(split_point):int(y2), int(x1):int(x2)]
                    
                    image_upper=cv2.resize(upper_part,(int(width),int(height/2)))
                    image_lower=cv2.resize(lower_part,(int(width),int(height/2)))

                    image_collage_horizontal =np.hstack([image_upper, image_lower])
                    image_filename = str(file_name) + ".jpg"  # Tên file ảnh đầu ra
                    cv2.imwrite("results/crop_image/" + image_filename, image_collage_horizontal)
    
                    # Tiếp tục xử lý ảnh chữ nhật cr_img ở đây
                    ocr_res = perform_ocr(image_collage_horizontal)
    
                    # Lưu hai phần ảnh
                    # cv2.imwrite("results/upper_part.jpg", upper_part)
                    # cv2.imwrite("results/" + str(ct) + ".jpg", lower_part)
                else:
                    # Biển số xe không gần vuông
                    # Xử lý ảnh bình thường ở đây (cr_img = img[int(y1):int(y2), int(x1):int(x2)])
                    cr_img = img[int(y1):int(y2), int(x1):int(x2)]
                    
                    
                    image_filename = str(file_name) + ".jpg"  # Tên file ảnh đầu ra
                    cv2.imwrite("results/crop_image/" + image_filename, cr_img)
                    ocr_res = perform_ocr(cr_img)
            recognized_text = ocr_res[0][0][0] if ocr_res else "No Text"
            print("recognized_text: ",recognized_text)
            
            ocr_conf = ocr_res[0][0][1] if ocr_res else "No Conf"
            ocr_conf = round(ocr_conf,3)
            print("ocr_conf: ",ocr_conf)
            # Draw on the image
            cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 2)  # Red bounding box
            cv2.putText(img, recognized_text, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
            cv2.putText(img, str(ocr_conf), (int(x1) + 150, int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

        # Writing output image for each image
        file_name = os.path.join(out_path, 'out_' + os.path.basename(input))
        cv2.imwrite(file_name, img)
    

def perform_ocr(image):
    ocr_res = ocr.ocr(image, cls=False, det=False)
    return ocr_res

In [89]:
import glob
# Usage example
files = glob.glob('./images/images_val/*.jpg', recursive = True)

x = 0
y = len(files)
for file in files:
    x = x + 1
    print(str(x) + "/" + str(y))
    test_img_yolov8(file, './results')

1/1662
file_name: boderngoaigiao0.jpg


image 1/1 /mnt/d/BKHN/alpr_project/learnopencv/images/images_val/boderngoaigiao0.jpg: 384x640 1 plate, 81.7ms
Speed: 313.2ms preprocess, 81.7ms inference, 4.3ms postprocess per image at shape (1, 3, 384, 640)


aspect_ratio: 4.353833


recognized_text:  41-291-NG-01
ocr_conf:  0.819
2/1662
file_name: boderngoaigiao1.jpg


image 1/1 /mnt/d/BKHN/alpr_project/learnopencv/images/images_val/boderngoaigiao1.jpg: 384x640 1 plate, 164.2ms
Speed: 4.0ms preprocess, 164.2ms inference, 6.3ms postprocess per image at shape (1, 3, 384, 640)



...

aspect_ratio: 1.3247019
recognized_text:  94-F1042.17
ocr_conf:  0.928
1661/1662
file_name: xemayBigPlate90.jpg


image 1/1 /mnt/d/BKHN/alpr_project/learnopencv/images/images_val/xemayBigPlate90.jpg: 480x640 1 plate, 133.0ms
Speed: 5.5ms preprocess, 133.0ms inference, 4.1ms postprocess per image at shape (1, 3, 480, 640)



aspect_ratio: 1.404232
recognized_text:  64-H69743
ocr_conf:  0.962
1662/1662
file_name: xemayBigPlate91.jpg


image 1/1 /mnt/d/BKHN/alpr_project/learnopencv/images/images_val/xemayBigPlate91.jpg: 480x640 1 plate, 133.4ms
Speed: 4.9ms preprocess, 133.4ms inference, 5.6ms postprocess per image at shape (1, 3, 480, 640)


aspect_ratio: 1.3883362
recognized_text:  67T34461
ocr_conf:  0.97


## Function for inferencing on videos

In [91]:
from ultralytics import YOLO
import cv2
import os
import time
import numpy as np 

# Load a model
model = YOLO("yolov8/alpr_yolov8n_8000img_100epochs.pt") 

def test_vid_yolov8(vid_dir, out_path):
    # Declaring variables for video processing.
    cap = cv2.VideoCapture(vid_dir)
    # Get the total frame count
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    codec = cv2.VideoWriter_fourcc(*'XVID')
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    # file_name = os.path.join(out_path, 'out_' + vid_dir.split('/')[-1] + )
    # out = cv2.VideoWriter(file_name, codec, fps, (width, height))
    out = cv2.VideoWriter(out_path, codec, fps, (width, height))

    # Frame count variable.
    ct = 0
    
    # Reading video frame by frame.
    while(cap.isOpened()):
        ret, img = cap.read()
        if ret == True:
            print(str(ct) + "/" + str(total_frames))

            # Noting time for calculating FPS.
            prev_time = time.time()

            # Use the model
            results = model(img)  # predict on an image
        
            for result in results:
                if result.boxes is not None and len(result.boxes) > 0:
                    # Get the bounding boxes and image for each result
                    bboxes = result.boxes[0].cpu().numpy()
                    # img = cv2.imread(input)  # Đảm bảo bạn đã định nghĩa biến 'input'
            
                    # Loop through bboxes and apply OCR, then draw on the image
                    for bbox in bboxes:
                        xyxy = bbox.xyxy
                        x1, y1, x2, y2 = xyxy[0]
            
                        # Kiểm tra xem biển số xe có gần vuông không (ví dụ: tỷ lệ 1:1)
                        width = x2 - x1
                        height = y2 - y1
                        aspect_ratio = width / height
                        print("aspect_ratio:", aspect_ratio)
            
                        if 0 <= aspect_ratio <= 1.5:
                            # Biển số xe gần vuông hoặc hình vuông
            
                            # Tính toán điểm chia ảnh thành hai phần trên và dưới
                            split_point = y1 + (y2 - y1) // 2

                            
                            # Tạo hai phần ảnh con từ ảnh cr_img
                            upper_part = img[int(y1):int(split_point), int(x1):int(x2)]
                            lower_part = img[int(split_point):int(y2), int(x1):int(x2)]
                            
                            image_upper=cv2.resize(upper_part,(int(width),int(height/2)))
                            image_lower=cv2.resize(lower_part,(int(width),int(height/2)))

                            image_collage_horizontal =np.hstack([image_upper, image_lower])
                            image_filename = str(ct) + ".jpg"  # Tên file ảnh đầu ra
                            cv2.imwrite("results/crop_image/" + image_filename, image_collage_horizontal)
            
                            # Tiếp tục xử lý ảnh chữ nhật cr_img ở đây
                            ocr_res = perform_ocr(image_collage_horizontal)
            
                            # Lưu hai phần ảnh
                            # cv2.imwrite("results/upper_part.jpg", upper_part)
                            # cv2.imwrite("results/" + str(ct) + ".jpg", lower_part)
                        else:
                            # Biển số xe không gần vuông
                            # Xử lý ảnh bình thường ở đây (cr_img = img[int(y1):int(y2), int(x1):int(x2)])
                            cr_img = img[int(y1):int(y2), int(x1):int(x2)]
                            image_filename = str(ct) + ".jpg"  # Tên file ảnh đầu ra
                            cv2.imwrite("results/crop_image/" + image_filename, cr_img)
                            ocr_res = perform_ocr(cr_img)
                        recognized_text = ocr_res[0][0][0] if ocr_res else "No Text"
                        print("recognized_text: ",recognized_text)
                        
                        ocr_conf = ocr_res[0][0][1] if ocr_res else "No Conf"
                        ocr_conf = round(ocr_conf,3)
                        print("ocr_conf: ",ocr_conf)
                        # Draw on the image
                        cv2.rectangle(img, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 2)  # Red bounding box
                        cv2.putText(img, recognized_text, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
                        cv2.putText(img, str(ocr_conf), (int(x1) + 150, int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
                else:
                    # If there are no bounding boxes or the result is empty, skip this result
                    continue

            # Calculating time taken and FPS for the whole process.
            tot_time = time.time() - prev_time
            fps = round(1 / tot_time,2)

            # Writing information onto the frame and saving it to be processed in a video.
            cv2.putText(img, 'frame: %d fps: %s' % (ct, fps),
                        (0, int(100 * 1)), cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 255), thickness=2)
            out.write(img)

            ct = ct + 1
        else:
            break
def perform_ocr(image):
    ocr_res = ocr.ocr(image, cls=False, det=False)
    return ocr_res

In [59]:
input_dir = 'test_video/test_5.mp4'
out_path = 'results/test_5.avi'
test_vid_yolov8(input_dir, out_path)

0/1162


0: 384x640 1 plate, 77.4ms
Speed: 300.4ms preprocess, 77.4ms inference, 4.4ms postprocess per image at shape (1, 3, 384, 640)



aspect_ratio: 2.51499
recognized_text:  BAY
ocr_conf:  0.313
1/1162


0: 384x640 1 plate, 161.1ms
Speed: 4.7ms preprocess, 161.1ms inference, 5.3ms postprocess per image at shape (1, 3, 384, 640)



aspect_ratio: 2.5208719
recognized_text:  Wye
ocr_conf:  0.258
2/1162


...

recognized_text:  53
ocr_conf:  0.583
1153/1162
1154/1162


0: 384x640 (no detections), 99.1ms
Speed: 4.1ms preprocess, 99.1ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 25.6ms
Speed: 2.7ms preprocess, 25.6ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 26.7ms
Speed: 2.7ms preprocess, 26.7ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 25.8ms
Speed: 2.7ms preprocess, 25.8ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 25.8ms
Speed: 2.7ms preprocess, 25.8ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 25.5ms
Speed: 2.4ms preprocess, 25.5ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)


1155/1162
1156/1162
1157/1162
1158/1162
1159/1162



0: 384x640 (no detections), 25.5ms
Speed: 2.6ms preprocess, 25.5ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 25.6ms
Speed: 2.6ms preprocess, 25.6ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)


1160/1162
1161/1162


## Function for integrating Tracker

Download pretrained deep association metric model called `mars-small128.pb`, can be downloaded from [here](https://drive.google.com/drive/folders/1n0jB3zwJysi6YDi4n0HVKz5yOZ0eNA2B?usp=sharing) and put under ./model_data/mars-small128.pb

In [75]:
import time
import numpy as np 

def tracker_test_vid(vid_dir, out_path):
    # Declaring variables for video processing.
    cap = cv2.VideoCapture(vid_dir)
    codec = cv2.VideoWriter_fourcc(*'XVID')
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    file_name = os.path.join(out_path, 'out_' + vid_dir.split('/')[-1])

    out = cv2.VideoWriter(file_name, codec, fps, (width, height))

    # Declaring variables for tracker.
    max_cosine_distance = 0.4
    nn_budget = None

    # Intializing tracker
    model_filename = 'model_data/mars-small128.pb'
    encoder = gdet.create_box_encoder(model_filename, batch_size=1)
    metric = nn_matching.NearestNeighborDistanceMetric("cosine", max_cosine_distance, nn_budget)
    tracker = Tracker(metric)

    # Initializing some helper variables.
    ct = 0
    preds = []
    total_obj = 0
    rec_tot_time = 1
    alpha = 0.5

    # Reading video frame by frame.
    while (cap.isOpened()):
        ret, img = cap.read()
        if ret == True:

            h, w = img.shape[:2]
            print(ct)

            w_scale = w / 1.55
            h_scale = h / 17

            # Method to blend two images, here used to make the information box transparent.
            overlay_img = img.copy()
            cv2.rectangle(img, (int(w_scale), 0), (w, int(h_scale * 3.4)), (0, 0, 0), -1)
            cv2.addWeighted(img, alpha, overlay_img, 1 - alpha, 0, overlay_img)

            # Noting time for calculating FPS.
            prev_time = time.time()

            # Use the model
            results = model(img)  # predict on an image

            for result in results:
                print("result", result)
                if result.boxes is not None and len(result.boxes) > 0:
                    # Get the bounding boxes and image for each result
                    bboxes = result.boxes[0].cpu().numpy()
                    if len(bboxes) > 0:
                        bboxes_np = []
                        for i in bboxes:
                            bbox = i.xyxy
                            bboxes_np.append(bbox)
                    # Convert the list of bounding boxes to a NumPy array
                    bboxes_np = np.array(bboxes_np)
                    print("bboxes_np", bboxes_np)
                    #     # Chuyển đổi bbox thành mảng NumPy
                    #     bbox = np.array(bbox)
                    # Getting appearence features of the object.
                    features = encoder(img, bboxes_np[0])
                    print("features", features)
                    # Storing all the required info in a list.
                    detections = [Detection(bbox, feature) for bbox, feature in
                                zip(bboxes_np, features)]

                    # Applying tracker.
                    # The tracker code flow: kalman filter -> target association(using hungarian algorithm) and appearance descriptor.
                    tracker.predict()
                    tracker.update(detections)
                    track_time = time.time() - prev_time

                    # Checking if tracks exist.
                    for track in tracker.tracks:
                        if not track.is_confirmed() or track.time_since_update > 1:
                            continue

                        # Changing track bbox to top left, bottom right coordinates
                        
                        # bbox = np.array(list(track.to_tlbr()))

                        # for i in range(len(bbox)):
                        #     if bbox[i] < 0:
                        #         bbox[i] = 0
                        bbox = [list(track.to_tlbr()) for track in tracker.tracks]
                        for i in range(len(bbox)):
                            for j in range(4):
                                if bbox[i][j] < 0:
                                    bbox[i][j] = 0
                        bbox = np.array(bbox)
                        # Extracting or cropping the license plate and applying the OCR.
                        cr_img = crop(img, bbox)

                        rec_pre_time = time.time()
                        result_ocr = ocr.ocr(cr_img, cls=False, det=False)
                        rec_tot_time = time.time() - rec_pre_time

                        ocr_res = result_ocr[0][0]
                        print(result_ocr)
                        rec_conf = result_ocr[0][1]

                        if rec_conf == 'nan':
                            rec_conf = 0

                        # Storing the ocr output for corresponding track id.
                        output_frame = {"track_id": track.track_id, "ocr_txt": ocr_res, "ocr_conf": rec_conf}

                        # Appending track_id to list only if it does not exist in the list.
                        if track.track_id not in list(set(ele['track_id'] for ele in preds)):
                            total_obj = total_obj + 1
                            preds.append(output_frame)
                        # Looking for the current track in the list and updating the highest confidence of it.
                        else:
                            preds, rec_conf, ocr_res = get_best_ocr(preds, rec_conf, ocr_res, track.track_id)

                        # Plotting the predictions using OpenCV.
                        txt = str(track.track_id) + '. ' + ocr_res
                        (label_width, label_height), baseline = cv2.getTextSize(txt, font, 2, 3)
                        top_left = tuple(map(int, [int(bbox[0]), int(bbox[1]) - (label_height + baseline)]))
                        top_right = tuple(map(int, [int(bbox[0]) + label_width, int(bbox[1])]))
                        org = tuple(map(int, [int(bbox[0]), int(bbox[1]) - baseline]))

                        cv2.rectangle(overlay_img, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])), blue_color, 2)
                        cv2.rectangle(overlay_img, top_left, top_right, blue_color, -1)
                        cv2.putText(overlay_img, txt, org, font, 2, white_color, 3)
                        #cv2.imwrite('/content/{}.jpg'.format(ct), img)
                    #cv2.imwrite('/content/{}.jpg'.format(ct), img)

            # Calculating time taken and FPS for the whole process.
            tot_time = time.time() - prev_time
            fps = 1 / tot_time

            # Writing information onto the frame and saving the frame to be processed into a video with title and values of different colors.
            if w < 2000:
                size = 1
            else:
                size = 2

            # Plotting frame count information on the frame.
            (label_width,label_height), baseline = cv2.getTextSize('Frame count:' , font,size,2)
            top_left = (int(w_scale) + 10, int(h_scale))
            cv2.putText(overlay_img, 'Frame count:', top_left, font, size, green_color, thickness=2)
            
            top_left_r1 = (int(w_scale) + 10 + label_width, int(h_scale))
            cv2.putText(overlay_img,'%d ' % (ct), top_left_r1, font, size, yellow_color, thickness=2)
            
            (label_width,label_height), baseline = cv2.getTextSize('Frame count:' + ' ' + str(ct) , font, size,2)
            top_left_r1 = (int(w_scale) + 10 + label_width, int(h_scale))
            cv2.putText(overlay_img, 'Total FPS:' , top_left_r1, font, size, green_color, thickness=2)
            
            (label_width,label_height), baseline = cv2.getTextSize('Frame count:' + ' ' + str(ct) + 'Total FPS:' , font, size,2)
            top_left_r1 = (int(w_scale) + 10 + label_width, int(h_scale))
            cv2.putText(overlay_img, '%s' % (int(fps)), top_left_r1, font, size, yellow_color, thickness=2)

            # Plotting Total FPS of ANPR information on the frame.
            cv2.putText(overlay_img, 'Detection FPS:', (top_left[0], int(h_scale * 1.7)), font, size, green_color, thickness=2)
            (label_width, label_height), baseline = cv2.getTextSize('Detection FPS:', font, size, 2)
            cv2.putText(overlay_img, '%d' % ((int(1 / det_time))), (top_left[0] + label_width, int(h_scale * 1.7)),
                        font, size, yellow_color, thickness=2)

            # Plotting Recognition/OCR FPS of ANPR on the frame.
            cv2.putText(overlay_img, 'Recognition FPS:', (top_left[0], int(h_scale * 2.42)), font, size, green_color,
                        thickness=2)
            (label_width, label_height), baseline = cv2.getTextSize('Recognition FPS:', font, size, 2)
            cv2.putText(overlay_img, '%s' % ((int(1 / rec_tot_time))), (top_left[0] + label_width, int(h_scale * 2.42)),
                        font, size, yellow_color, thickness=2)
            cv2.imwrite('/content/{}.jpg'.format(ct), overlay_img)
            out.write(overlay_img)

            # Increasing frame count.
            ct = ct + 1
        else:
            break



In [76]:
# def tracker_test_vid(vid_dir, config_file, weights,out_path):
#   # Declaring variables for video processing.
#   cap = cv2.VideoCapture(vid_dir)
#   codec = cv2.VideoWriter_fourcc(*'XVID')
#   width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
#   height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
#   fps = int(cap.get(cv2.CAP_PROP_FPS))
#   file_name = os.path.join(out_path, 'out_' + vid_dir.split('/')[-1])

#   out = cv2.VideoWriter(file_name, codec, fps, (width, height))

#   # Declaring variables for tracker.
#   max_cosine_distance = 0.4
#   nn_budget = None

#   # Intializing tracker
#   model_filename = './model_data/mars-small128.pb'
#   encoder = gdet.create_box_encoder(model_filename, batch_size=1)
#   metric = nn_matching.NearestNeighborDistanceMetric("cosine", max_cosine_distance, nn_budget)
#   tracker = Tracker(metric)

#   # Initializing some helper variables.
#   ct = 0
#   preds = []
#   total_obj = 0
#   rec_tot_time = 1
#   alpha = 0.5

#   # Reading video frame by frame.
#   while(cap.isOpened()):
#     ret, img = cap.read()
#     if ret == True:

#         h, w = img.shape[:2]
#         print(ct)

#         w_scale = w/1.55
#         h_scale = h/17

#         # Method to blend two images, here used to make the information box transparent.
#         overlay_img = img.copy()
#         cv2.rectangle(img, (int(w_scale), 0), (w, int(h_scale*3.4)), (0,0,0), -1)
#         cv2.addWeighted(img, alpha, overlay_img, 1 - alpha, 0, overlay_img)

#         # Noting time for calculating FPS.
#         prev_time = time.time()

        # # Performing the YOLOv4 detection.
        # bboxes, scores, det_time = yolo_det(img, config_file, data_file, batch_size, weights, thresh, out_path, network, class_names, class_colors)

        # if list(bboxes):
        #   # Getting appearence features of the object.
        #   features = encoder(img, bboxes)
        #   # Storing all the required info in a list.
        #   detections = [Detection(bbox, score, feature) for bbox, score, feature in zip(bboxes, scores, features)]

#           # Applying tracker.
#           # The tracker code flow: kalman filter -> target association(using hungarian algorithm) and appearance descriptor.
#           tracker.predict()
#           tracker.update(detections)
#           track_time = time.time() - prev_time

#           # Checking if tracks exist.
#           for track in tracker.tracks:
#             if not track.is_confirmed() or track.time_since_update > 1:
#                 continue

#             # Changing track bbox to top left, bottom right coordinates
#             bbox = list(track.to_tlbr())

#             for i in range(len(bbox)):
#               if bbox[i] < 0:
#                 bbox[i] = 0

#             # Extracting or cropping the license plate and applying the OCR.
#             cr_img = crop(img, bbox)

#             rec_pre_time = time.time()
#             result = ocr.ocr(cr_img, cls=False, det=False)
#             rec_tot_time = time.time() - rec_pre_time

#             ocr_res = result[0][0]
#             print(result)
#             rec_conf = result[0][1]

#             if rec_conf == 'nan':
#               rec_conf = 0

#             # Storing the ocr output for corresponding track id.
#             output_frame = {"track_id":track.track_id, "ocr_txt":ocr_res, "ocr_conf":rec_conf}

#             # Appending track_id to list only if it does not exist in the list.
#             if track.track_id not in list(set(ele['track_id'] for ele in preds)):
#               total_obj = total_obj + 1
#               preds.append(output_frame)
#             # Looking for the current track in the list and updating the highest confidence of it.
#             else:
#               preds, rec_conf, ocr_res = get_best_ocr(preds, rec_conf, ocr_res, track.track_id)

#             # Plotting the predictions using OpenCV.
#             txt = str(track.track_id) + '. ' + ocr_res
#             (label_width,label_height), baseline = cv2.getTextSize(txt , font,2,3)
#             top_left = tuple(map(int,[int(bbox[0]),int(bbox[1])-(label_height+baseline)]))
#             top_right = tuple(map(int,[int(bbox[0])+label_width,int(bbox[1])]))
#             org = tuple(map(int,[int(bbox[0]),int(bbox[1])-baseline]))

#             cv2.rectangle(overlay_img, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])), blue_color, 2)
#             cv2.rectangle(overlay_img, top_left, top_right, blue_color, -1)
#             cv2.putText(overlay_img,txt, org, font, 2, white_color, 3)
#             #cv2.imwrite('/content/{}.jpg'.format(ct), img)

#           # Calculating time taken and FPS for the whole process.
#           tot_time = time.time() - prev_time
#           fps = 1/tot_time

#           # Writing information onto the frame and saving the frame to be processed into a video with title and values of different colors.
#           if w < 2000:
#             size = 1
#           else:
#             size = 2

#           # Plotting frame count information on the frame.
#           (label_width,label_height), baseline = cv2.getTextSize('Frame count:' , font,size,2)
#           top_left = (int(w_scale) + 10, int(h_scale))
#           cv2.putText(overlay_img, 'Frame count:', top_left, font, size, green_color, thickness=2)

#           top_left_r1 = (int(w_scale) + 10 + label_width, int(h_scale))
#           cv2.putText(overlay_img,'%d ' % (ct), top_left_r1, font, size, yellow_color, thickness=2)

#           (label_width,label_height), baseline = cv2.getTextSize('Frame count:' + ' ' + str(ct) , font, size,2)
#           top_left_r1 = (int(w_scale) + 10 + label_width, int(h_scale))
#           cv2.putText(overlay_img, 'Total FPS:' , top_left_r1, font, size, green_color, thickness=2)

#           (label_width,label_height), baseline = cv2.getTextSize('Frame count:' + ' ' + str(ct) + 'Total FPS:' , font, size,2)
#           top_left_r1 = (int(w_scale) + 10 + label_width, int(h_scale))
#           cv2.putText(overlay_img, '%s' % (int(fps)), top_left_r1, font, size, yellow_color, thickness=2)

#           # Plotting Total FPS of ANPR information on the frame.
#           cv2.putText(overlay_img, 'Detection FPS:' ,(top_left[0], int(h_scale*1.7)), font, size, green_color, thickness=2)
#           (label_width,label_height), baseline = cv2.getTextSize('Detection FPS:', font,size,2)
#           cv2.putText(overlay_img, '%d' % ((int(1/det_time))),(top_left[0] + label_width, int(h_scale*1.7)), font, size, yellow_color, thickness=2)

#           # Plotting Recognition/OCR FPS of ANPR on the frame.
#           cv2.putText(overlay_img, 'Recognition FPS:',(top_left[0], int(h_scale*2.42)), font, size, green_color, thickness=2)
#           (label_width,label_height), baseline = cv2.getTextSize('Recognition FPS:', font,size,2)
#           cv2.putText(overlay_img, '%s' % ((int(1/rec_tot_time))),(top_left[0] + label_width, int(h_scale*2.42)), font, size, yellow_color, thickness=2)
#           cv2.imwrite('/content/{}.jpg'.format(ct), overlay_img)
#           out.write(overlay_img)

#         # Increasing frame count.
#         ct = ct + 1
#     else:
#       break

**Test Vid**

In [77]:
input_dir = 'test_video/test_2.mp4'
out_path = 'results/test_2.avi'

In [78]:
tracker_test_vid(input_dir, out_path)

OpenCV: FFMPEG: tag 0x44495658/'XVID' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'



0


0: 384x640 1 plate, 78.1ms
Speed: 306.4ms preprocess, 78.1ms inference, 5.2ms postprocess per image at shape (1, 3, 384, 640)


result ultralytics.yolo.engine.results.Results object with attributes:

boxes: ultralytics.yolo.engine.results.Boxes object
keypoints: None
keys: ['boxes']
masks: None
names: {0: 'plate'}
orig_img: array([[[ 39,  59,  61],
        [102, 122, 124],
        [ 35,  55,  57],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[ 82, 102, 104],
        [148, 168, 170],
        [ 42,  62,  64],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       [[ 17,  37,  39],
        [ 27,  47,  49],
        [ 42,  62,  64],
        ...,
        [  0,   0,   0],
        [  0,   0,   0],
        [  0,   0,   0]],

       ...,

       [[148, 156, 160],
        [214, 222, 226],
        [229, 237, 241],
        ...,
        [ 77,  86,  87],
        [ 78,  87,  88],
        [ 77,  86,  87]],

       [[148, 156, 160],
        [214, 222, 226],
        [229, 237, 241],
        ...,
        [ 78,  87,  88],
        [ 79,  8

TypeError: Detection.__init__() missing 1 required positional argument: 'feature'

In [1]:
# DeepSORT imports.
%cd ./deep_sort

/mnt/d/BKHN/alpr_project/learnopencv/deep_sort


In [2]:
from application_util import preprocessing
from deep_sort import nn_matching
from deep_sort.detection import Detection
from deep_sort.tracker import Tracker
from tools_deepsort import generate_detections as gdet
import uuid

2023-09-24 12:58:17.993153: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-24 12:58:18.063267: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-24 12:58:18.064564: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-24 12:58:19.183524: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
# DeepSORT imports.
%cd ../

/mnt/d/BKHN/alpr_project/learnopencv


In [17]:
from ultralytics import YOLO
import cv2
import os


import time
import numpy as np

from paddleocr import PaddleOCR
ocr = PaddleOCR(lang='en',rec_algorithm='CRNN')

# Load a model
model = YOLO("yolov8/alpr_yolov8n_8000img_100epochs.pt") 

# Initialize DeepSORT tracker
max_cosine_distance = 0.4
nn_budget = None
model_filename = './model_data/mars-small128.pb'
encoder = gdet.create_box_encoder(model_filename, batch_size=1)
metric = nn_matching.NearestNeighborDistanceMetric("cosine", max_cosine_distance, nn_budget)
tracker = Tracker(metric)

def test_video_yolov8(input, out_path):
    # Declaring variables for video processing.
    cap = cv2.VideoCapture(input_dir)
    codec = cv2.VideoWriter_fourcc(*'XVID')
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    out = cv2.VideoWriter(out_path, codec, fps, (width, height))

    # Frame count variable.
    ct = 0

    # Initialize DeepSORT tracker
    max_cosine_distance = 0.4
    nn_budget = None
    model_filename = './model_data/mars-small128.pb'
    encoder = gdet.create_box_encoder(model_filename, batch_size=1)
    metric = nn_matching.NearestNeighborDistanceMetric("cosine", max_cosine_distance, nn_budget)
    tracker = Tracker(metric)
    
    # Reading video frame by frame.
    while(cap.isOpened()):
        ret, img = cap.read()
        if ret == True:
            print(ct)

            # Noting time for calculating FPS.
            prev_time = time.time()

            # Use the model
            results = model(img)  # predict on an image

            bboxes_np = []  # List to store bounding boxes
            # Filter out low-confidence OCR results
            detections = []
            for result in results:
                if result.boxes is not None and len(result.boxes) > 0:
                    # Get the bounding boxes for each result
                    bboxes = result.boxes[0].cpu().numpy()
                    for bbox in bboxes:
                        xyxy = bbox.xyxy[0]
                        x1, y1, x2, y2 = xyxy
                        
                        # Crop license plate region
                        cr_img = img[int(y1):int(y2), int(x1):int(x2)]
                        
                        # Perform OCR (you'll need to add your OCR code here)
                        ocr_res = perform_ocr(cr_img)
                        

                        if ocr_res and ocr_res[0][0][1] > 0.6:
                            # Append bounding box to the list
                            bboxes_np.append(xyxy)
                            bboxes_np = np.array(bboxes_np)
                            # Get appearance feature for this bounding box
                            confidence = ocr_res[0][0][1] 
                            feature = encoder(img, [xyxy])[0]
                            # Create a Detection object with both bbox and feature
                            detection = Detection(xyxy, confidence, feature)
                            
                            # Append the detection to your list of detections or use it as needed
                            detections.append(detection)
            # if len(bboxes_np) > 0:
                # # Convert the list of bounding boxes to a NumPy array
                # bboxes_np = np.array(bboxes_np)
                # print("bboxes_np:", bboxes_np)
                # # Get appearance features of the objects
                # features = encoder(img, bboxes_np)
                # print("features:", features)
                # # Create a list of Detection objects
                # detections = [Detection(bbox, feature) for bbox, feature in zip(bboxes_np, features)]
    
                # Apply the tracker
                tracker.predict()
                tracker.update(detections)
    
                # Loop through the tracked objects
                for track in tracker.tracks:
                    if not track.is_confirmed() or track.time_since_update > 1:
                        continue
    
                    # Get the track's bounding box
                    bbox = track.to_tlbr()
                    x1, y1, x2, y2 = bbox.astype(int)
    
                    # Crop license plate region
                    cr_img = img[y1:y2, x1:x2]
                    
                    # Check if the cropped image is empty or invalid
                    if cr_img is None or cr_img.size == 0:
                        continue
    
                    # Perform OCR (you'll need to add your OCR code here)
                    ocr_res = perform_ocr(cr_img)
    
                    recognized_text = ocr_res[0][0][0] if ocr_res else "No Text"
                    ocr_conf = ocr_res[0][0][1] if ocr_res else "No Conf"
                    ocr_conf = round(ocr_conf, 3)
    
                    # Draw on the image
                    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), 2)  # Red bounding box
                    cv2.putText(img, recognized_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
                    cv2.putText(img, str(ocr_conf), (x1 + 100, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
    
                # Calculating time taken and FPS for the whole process.
                tot_time = time.time() - prev_time
                fps = round(1 / tot_time, 2)
    
                # Writing information onto the frame and saving it to be processed in a video.
                cv2.putText(img, 'frame: %d fps: %s' % (ct, fps),
                            (0, int(100 * 1)), cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 255), thickness=2)
                out.write(img)
    
                ct = ct + 1
        else:
            break

def perform_ocr(image):
    ocr_res = ocr.ocr(image, cls=False, det=False)
    return ocr_res

[2023/09/24 13:12:29] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, gpu_id=0, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='/root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='CRNN', rec_model_dir='/root/.paddleocr/whl/rec/en/en_PP-OCRv4_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, rec_char_dict_path='/usr/l

In [18]:
input_dir = 'test_video/test_2.mp4'
out_path = 'results/test_2.avi'
test_video_yolov8(input_dir, out_path)

0



0: 384x640 1 plate, 79.2ms
Speed: 3.9ms preprocess, 79.2ms inference, 3.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 plates, 163.1ms
Speed: 4.4ms preprocess, 163.1ms inference, 5.2ms postprocess per image at shape (1, 3, 384, 640)


1



0: 384x640 1 plate, 164.5ms
Speed: 5.6ms preprocess, 164.5ms inference, 5.0ms postprocess per image at shape (1, 3, 384, 640)


2


...

526
527
528
529
530


0: 384x640 (no detections), 26.2ms
Speed: 3.0ms preprocess, 26.2ms inference, 2.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 26.0ms
Speed: 3.1ms preprocess, 26.0ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 26.2ms
Speed: 3.2ms preprocess, 26.2ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 26.6ms
Speed: 3.1ms preprocess, 26.6ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 26.0ms
Speed: 3.3ms preprocess, 26.0ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 25.9ms


531
532
533
534
535


Speed: 3.1ms preprocess, 25.9ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 25.9ms
Speed: 3.3ms preprocess, 25.9ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 25.8ms
Speed: 2.7ms preprocess, 25.8ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)


536
537
